In [1]:
import pandas as pd

quota = pd.read_html('quota.html', match= "Quota")

In [2]:
print(len(quota))

1


In [3]:
df = quota[0]
df.head()

,0,1,2,3,4
0,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,Department,Statu,Quota,Current,NaN
2,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71,NaN
3,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1,NaN


In [4]:
df.columns = ['Department', 'Status', "Quota", "Current","a"]
df.head()

,Department,Status,Quota,Current,a
0,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,Department,Statu,Quota,Current,NaN
2,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71,NaN
3,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1,NaN


In [5]:
df.drop("a",axis=1, inplace=True)
df

,Department,Status,Quota,Current
0,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,Department,Statu,Quota,Current
2,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71
3,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1


In [6]:
df.drop(labels=range(0,2), inplace=True)
df = df.reset_index(drop=True)
df

,Department,Status,Quota,Current
0,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71
1,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1


In [7]:
# import requests
# from bs4 import BeautifulSoup
 
 
# url = 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=MIS&bolum=MANAGEMENT+INFORMATION+SYSTEMS'
# reqs = requests.get(url)
# soup = BeautifulSoup(reqs.text, 'html.parser')
 
# urls = []
# for link in soup.find_all('a'):
#     print(link.get('href'))

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Chrome()
driver.get("https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=MIS&bolum=MANAGEMENT+INFORMATION+SYSTEMS")
info_link = driver.find_element(By.LINK_TEXT,"Info")
info_link.click()